#ASSIGNMENT - A2b
# **Performing Multiple regression analysis on IPL_ball_by_ball data**

* **AUTHOR**     : Rakshitha Vignesh Sargurunathan               

* **VID**        : V01109007

* **CREATED ON** : 06/22/2024

# IPL DATA :

In [21]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

In [7]:
ipl = pd.read_csv("/content/drive/MyDrive/SCMA/A1b/IPL_ball_by_ball_updated till 2024.csv",low_memory=False)
salary = pd.read_excel("/content/drive/MyDrive/SCMA/A1b/IPL SALARIES 2024.xlsx")


In [8]:
ipl.columns

Index(['Match id', 'Date', 'Season', 'Batting team', 'Bowling team',
       'Innings No', 'Ball No', 'Bowler', 'Striker', 'Non Striker',
       'runs_scored', 'extras', 'type of extras', 'score', 'score/wicket',
       'wicket_confirmation', 'wicket_type', 'fielders_involved',
       'Player Out'],
      dtype='object')

In [9]:
grouped_data = ipl.groupby(['Season', 'Innings No', 'Striker','Bowler']).agg({'runs_scored': sum, 'wicket_confirmation':sum}).reset_index()
grouped_data

,Season,Innings No,Striker,Bowler,runs_scored,wicket_confirmation
0,2007/08,1,A Chopra,DP Vijaykumar,1,0
1,2007/08,1,A Chopra,DW Steyn,1,1
2,2007/08,1,A Chopra,GD McGrath,2,0
3,2007/08,1,A Chopra,PJ Sangwan,6,1
4,2007/08,1,A Chopra,RP Singh,9,0
...,...,...,...,...,...,...
48781,2024,2,YBK Jaiswal,RJW Topley,0,1
48782,2024,2,YBK Jaiswal,SM Curran,6,0
48783,2024,2,YBK Jaiswal,Tilak Varma,5,0
48784,2024,2,YBK Jaiswal,VG Arora,10,1


In [10]:
total_runs_each_year = grouped_data.groupby(['Season', 'Striker'])['runs_scored'].sum().reset_index()
total_wicket_each_year = grouped_data.groupby(['Season', 'Bowler'])['wicket_confirmation'].sum().reset_index()

In [11]:
total_runs_each_year

,Season,Striker,runs_scored
0,2007/08,A Chopra,42
1,2007/08,A Kumble,13
2,2007/08,A Mishra,37
3,2007/08,A Mukund,0
4,2007/08,A Nehra,3
...,...,...,...
2593,2024,Vijaykumar Vyshak,1
2594,2024,WG Jacks,176
2595,2024,WP Saha,135
2596,2024,Washington Sundar,0


In [12]:
total_wicket_each_year

,Season,Bowler,wicket_confirmation
0,2007/08,A Kumble,8
1,2007/08,A Mishra,11
2,2007/08,A Nehra,14
3,2007/08,A Nel,1
4,2007/08,A Symonds,0
...,...,...,...
1924,2024,WG Jacks,2
1925,2024,Washington Sundar,1
1926,2024,YS Chahal,14
1927,2024,Yash Dayal,8


In [13]:
!pip install fuzzywuzzy


In [14]:
!pip install python-Levenshtein

In [15]:
from fuzzywuzzy import process

# Convert to DataFrame
df_salary = salary.copy()
df_runs = total_runs_each_year.copy()

# Function to match names
def match_names(name, names_list):
    match, score = process.extractOne(name, names_list)
    return match if score >= 80 else None  # Use a threshold score of 80

In [16]:
# Create a new column in df_salary with matched names from df_runs
df_salary['Matched_Player'] = df_salary['Player'].apply(lambda x: match_names(x, df_runs['Striker'].tolist()))


In [17]:
# Merge the DataFrames on the matched names
df_merged = pd.merge(df_salary, df_runs, left_on='Matched_Player', right_on='Striker')
df_original = df_merged.copy()

In [18]:
#susbsets data for last three years
df_merged = df_merged.loc[df_merged['Season'].isin(['2021', '2022', '2023'])]
df_merged.Season.unique()

array(['2023', '2022', '2021'], dtype=object)

In [19]:
df_merged.head()

,Player,Salary,Rs,international,iconic,Matched_Player,Season,Striker,runs_scored
0,Abhishek Porel,20 lakh,20,0,NaN,Abishek Porel,2023,Abishek Porel,33
3,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2022,A Nortje,1
4,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2023,A Nortje,37
13,Axar Patel,9 crore,900,0,NaN,AR Patel,2021,AR Patel,40
14,Axar Patel,9 crore,900,0,NaN,AR Patel,2022,AR Patel,182


In [23]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_percentage_error

X = df_merged[['runs_scored']] # Independent variable(s)
y = df_merged['Rs'] # Dependent variable
# Split the data into training and test sets (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Create a LinearRegression model
model = LinearRegression()

In [24]:
# Fit the model on the training data
model.fit(X_train, y_train)

LinearRegression()

In [25]:
X.head()

,runs_scored
0,33
3,1
4,37
13,40
14,182


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

X = df_merged[['runs_scored']] # Independent variable(s)
y = df_merged['Rs'] # Dependent variable

# Split the data into training and test sets (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
# Add a constant to the model (intercept)
X_train_sm = sm.add_constant(X_train)

# Create a statsmodels OLS regression model
model = sm.OLS(y_train, X_train_sm).fit()

# Get the summary of the model
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                     Rs   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     15.83
Date:                Sun, 23 Jun 2024   Prob (F-statistic):           0.000100
Time:                        17:28:20   Log-Likelihood:                -1379.8
No. Observations:                 183   AIC:                             2764.
Df Residuals:                     181   BIC:                             2770.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         430.8473     46.111      9.344      

In [28]:
from fuzzywuzzy import process

# Convert to DataFrame
df_salary = salary.copy()
df_runs = total_wicket_each_year.copy()

# Function to match names
def match_names(name, names_list):
    match, score = process.extractOne(name, names_list)
    return match if score >= 80 else None  # Use a threshold score of 80

# Create a new column in df_salary with matched names from df_runs
df_salary['Matched_Player'] = df_salary['Player'].apply(lambda x: match_names(x, df_runs['Bowler'].tolist()))


In [29]:
# Merge the DataFrames on the matched names
df_merged = pd.merge(df_salary, df_runs, left_on='Matched_Player', right_on='Bowler')
df_merged[df_merged['wicket_confirmation']>10]

,Player,Salary,Rs,international,iconic,Matched_Player,Season,Bowler,wicket_confirmation
1,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2020/21,A Nortje,23
2,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2021,A Nortje,12
4,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2023,A Nortje,11
6,Axar Patel,9 crore,900,0,NaN,AR Patel,2014,AR Patel,19
7,Axar Patel,9 crore,900,0,NaN,AR Patel,2015,AR Patel,14
...,...,...,...,...,...,...,...,...,...
589,T. Natarajan,3.2 crore,320,0,NaN,T Natarajan,2020/21,T Natarajan,19
591,T. Natarajan,3.2 crore,320,0,NaN,T Natarajan,2022,T Natarajan,20
592,T. Natarajan,3.2 crore,320,0,NaN,T Natarajan,2023,T Natarajan,13
593,T. Natarajan,3.2 crore,320,0,NaN,T Natarajan,2024,T Natarajan,13


In [30]:
#susbsets data for last three years
df_merged = df_merged.loc[df_merged['Season'].isin(['2022'])]
df_merged.Season.unique()

array(['2022'], dtype=object)

In [31]:
# Assuming df_merged is already defined and contains the necessary columns
X = df_merged[['wicket_confirmation']] # Independent variable(s)
y = df_merged['Rs'] # Dependent variable

# Split the data into training and test sets (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add a constant to the model (intercept)
X_train_sm = sm.add_constant(X_train)

# Create a statsmodels OLS regression model
model = sm.OLS(y_train, X_train_sm).fit()

# Get the summary of the model
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                     Rs   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   F-statistic:                     3.688
Date:                Sun, 23 Jun 2024   Prob (F-statistic):             0.0610
Time:                        17:28:37   Log-Likelihood:                -360.96
No. Observations:                  48   AIC:                             725.9
Df Residuals:                      46   BIC:                             729.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 396.6881    

**INFERENCE** :

* R² score of 0.074 indicates that approximately 7.4% of the variance in the dependent variable (salary) can be explained by the independent variable (runs scored).

* The positive coefficient for runs scored (0.6895) suggests that as the number of runs scored by a player increases, their salary also tends to increase. This aligns with the expectation that better performance (more runs) would lead to higher salaries.

* The R² score of 0.074 is relatively low, indicating that runs scored alone do not explain much of the variability in salaries. This suggests that other factors (e.g., player experience, position, team performance, endorsements, etc.) also play significant roles in determining a player's salary.

**CONCLUSION**

The analysis indicates a statistically significant positive relationship between runs scored and player salary, but the low R² score suggests that many other factors are also at play. Runs scored is a significant but not a comprehensive predictor of salary. Further research incorporating additional variables is recommended to build a more robust model for predicting player salaries.





